In [4]:
# ===== Reduced Order Modeling with NGSolve + pyMOR (POD-Galerkin) =====
from ngsolve import *
from ngsolve import Mesh
from ngsolve import x, y, z
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square
from netgen.occ import *

from pymor.algorithms.pod import pod
from pymor.vectorarrays.numpy import NumpyVectorSpace
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np

In [5]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1, quad_dominated=True))
order = 2
V = H1(mesh,order=order,dirichlet="left")

E = 5e6 # Youngs module
nu = 0.25 # Poisson ratio
CDR = [-0.3,0.3] # Corner displacement range
rho = 7850 # Material density
lam = E*nu / ((1+nu)*(1-2*nu))
G  = E / (2*(1+nu))
g = 9.81 # graviational acceleration

In [6]:
# def phi(x,y,dx,dy):
#     return [x+x*y*dx,y+x*y*dy]

# def eps(u):
#     return 0.5 * (grad(u) + grad(u).trans)

# def sigma(u):
#     return 2*G*eps(u) + lam*Trace(eps(u))*Id(u.dim)

# u, v = V.TnT()
# a = BilinearForm(V)
# a+= InnerProduct(sigma(u), eps(v)) * dx

# f = LinearForm(V).Add( CoefficientFunction((0,-g*rho)) * v * dx )


In [7]:
from ngsolve import *
from netgen.geom2d import unit_square

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

order = 2
V = VectorH1(mesh, order=order, dirichlet="left")

E, nu = 5e6, 0.25
lam = E*nu / ((1+nu)*(1-2*nu))
mu  = E/(2*(1+nu))

u, v = V.TnT()

def eps(u):
    return 0.5*(Grad(u) + Grad(u).trans)

def sigma(u):
    return 2*mu*eps(u) + lam*Trace(eps(u))*Id(u.dim)

a = BilinearForm(V).Add( InnerProduct(sigma(u), eps(v)) * dx )

rho, g = 7850, 9.81
f = LinearForm(V).Add( CoefficientFunction((0,-g*rho)) * v * dx )

gfu = GridFunction(V)
a.Assemble(); f.Assemble()
# solvers.CG(a.mat, gfu.vec, f.vec)
inv = a.mat.Inverse(V.FreeDofs(), inverse="pardiso")
gfu.vec.data = inv * f.vec


In [14]:
from ngsolve.webgui import Draw

Draw(gfu,mesh)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [11]:
from ngsolve.internal import visoptions
Draw(gfu)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene